<a href="https://colab.research.google.com/github/JoanChege/chatbot/blob/main/hf_dataset_chabot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyyaml
!pip install nltk
#!pip install chatterbot
!pip install datasets



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

LOAD THE DATASET

In [4]:
from datasets import load_dataset

dataset = load_dataset('Amod/mental_health_counseling_conversations')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [19]:
import pandas as pd

data_list = []
for conversation in dataset['train']:
    context = conversation['Context']
    response = conversation['Response']
    data_list.append({'context': context, 'responses': response})

df = pd.DataFrame(data_list)
print(df)


                                                context  \
0     I'm going through some things with my feelings...   
1     I'm going through some things with my feelings...   
2     I'm going through some things with my feelings...   
3     I'm going through some things with my feelings...   
4     I'm going through some things with my feelings...   
...                                                 ...   
3507  My grandson's step-mother sends him to school ...   
3508  My boyfriend is in recovery from drug addictio...   
3509  The birth mother attempted suicide several tim...   
3510  I think adult life is making him depressed and...   
3511  I just took a job that requires me to travel f...   

                                              responses  
0     If everyone thinks you're worthless, then mayb...  
1     Hello, and thank you for your question and see...  
2     First thing I'd suggest is getting the sleep y...  
3     Therapy is essential for those that are feelin...  
4

CLEAN THE DATASET

In [32]:
# Lowercase for case-insensitive cleaning
df['context'] = df['context'].str.lower()
df['responses'] = df['responses'].str.lower()

# Remove punctuation using a regular expression
import re
punc_pattern = r'[^\w\s]'  # Matches any character that's not alphanumeric or whitespace
df['context'] = df['context'].str.replace(punc_pattern, '', regex=True)
df['responses'] = df['responses'].str.replace(punc_pattern, '', regex=True)

# Remove extra spaces with regular expression
whitespace_pattern = r'\s+'  # Matches one or more whitespace characters
df['context'] = df['context'].str.replace(whitespace_pattern, ' ', regex=True)
df['responses'] = df['responses'].str.replace(whitespace_pattern, ' ', regex=True)

# Optional: Remove stopwords (common words like "the", "a", "an")
from nltk.corpus import stopwords  # Import stopwords library
stop_words = stopwords.words('english')

def remove_stopwords(text):
  return [word for word in text.split() if word not in stop_words]

df['context'] = df['context'].apply(remove_stopwords)
df['responses'] = df['responses'].apply(remove_stopwords)

print(df)


                                                context  \
0     [im, going, things, feelings, barely, sleep, n...   
1     [im, going, things, feelings, barely, sleep, n...   
2     [im, going, things, feelings, barely, sleep, n...   
3     [im, going, things, feelings, barely, sleep, n...   
4     [im, going, things, feelings, barely, sleep, n...   
...                                                 ...   
3507  [grandsons, stepmother, sends, school, pink, b...   
3508  [boyfriend, recovery, drug, addiction, recentl...   
3509  [birth, mother, attempted, suicide, several, t...   
3510  [think, adult, life, making, depressed, often,...   
3511  [took, job, requires, travel, far, away, home,...   

                                              responses  
0     [everyone, thinks, youre, worthless, maybe, ne...  
1     [hello, thank, question, seeking, advice, feel...  
2     [first, thing, id, suggest, getting, sleep, ne...  
3     [therapy, essential, feeling, depressed, worth...  
4

FEATURE EXTRACTION AND BASIC ENGINEERING

In [33]:
from nltk.util import ngrams
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
#extract features from the cleaned data
def extract_features(cleaned_convos):
  featured_data=[]
  for conv in cleaned_convos:
      category = conv['category']
      conversation = conv['conversation']
      features = []
      for utterance in conversation:
        word_tokens = utterance.split()  # tokenization was already done in clean_data

        # Feature Extraction with N-grams and POS tags
        bigrams = ngrams(word_tokens, 2)  # Extract bigrams (2-word sequences)
        pos_tags = pos_tag(word_tokens)  # Get POS tags for each word

        # Add features based on extracted data
        features.extend(bigrams)
        features.extend(pos_tags)

        #basic feature engineering
        features.append(len(word_tokens)) #word count in the utterance
        features.append(sum(len(word) for word in word_tokens) / len(word_tokens))

      featured_data.append({
          'category': category,
          'conversation': conversation,
          'features': features
      })
  return featured_data
data_features = extract_features(cleaned_data)
cleaned_features_df = pd.DataFrame(data_features)
print(cleaned_features_df)


      category                                       conversation  \
0   depression  [depression, depression common serious medical...   
1   depression  [feel let parents, matter whatyour parents alw...   
2   depression  [types depression, many different forms depres...   
3   depression  [good nothing, dont ever think achieve want, c...   
4   depression  [feeling stressed lately, research says music ...   
5   depression  [want kill, helpline availableyou contact coun...   
6   depression  [afraid fail, failure stepping stone towards s...   
7   depression  [depression, depression mood disorder causes p...   
8   depression  [causes depression, complicated multiple cause...   
9   depression  [top causes depression, abuse past physical se...   
10  depression  [age group highest rate depression, number rep...   
11  depression  [country highest rate depression, depressed co...   
12  depression  [country lowest rate depression, depressed cou...   
13  depression  [school cause depr

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert conversations to text data
text_data = [' '.join(conv) for conv in cleaned_features_df['conversation']]

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Convert text data to TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(text_data)

# Print the shape of the TF-IDF matrix
print("TF-IDF matrix shape:", X_tfidf.shape)

TF-IDF matrix shape: (51, 619)


CLUSTERING- KMEANS


In [ ]:
from sklearn.cluster import KMeans

# Specify the number of clusters
num_clusters = 5  # Adjust as needed

# Apply K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, n_init=10)
kmeans.fit(X_tfidf)

# Get cluster labels
cluster_labels = kmeans.labels_

# Add cluster labels to the DataFrame
cleaned_features_df['cluster'] = cluster_labels

# Print the first few rows of the DataFrame with cluster labels
print(cleaned_features_df.head())


     category                                       conversation  \
0  depression  [depression, depression common serious medical...   
1  depression  [feel let parents, matter whatyour parents alw...   
2  depression  [types depression, many different forms depres...   
3  depression  [good nothing, dont ever think achieve want, c...   
4  depression  [feeling stressed lately, research says music ...   

                                            features  cluster  
0  [(depression, NN), 1, 10.0, (depression, commo...        2  
1  [(feel, let), (let, parents), (feel, VB), (let...        3  
2  [(types, depression), (types, NNS), (depressio...        2  
3  [(good, nothing), (good, JJ), (nothing, NN), 2...        0  
4  [(feeling, stressed), (stressed, lately), (fee...        3  
